<a href="https://colab.research.google.com/github/SaiLikhithaNM/Active-Drowning-Detection/blob/main/DROWNING_DETECTION.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from keras.models import load_model  # TensorFlow is required for Keras to work
from PIL import Image, ImageOps  # Install pillow instead of PIL
import numpy as np

# Disable scientific notation for clarity
np.set_printoptions(suppress=True)

# Load the model
model = load_model("keras_model.h5", compile=False)

# Load the labels
class_names = open("labels.txt", "r").readlines()

# Create the array of the right shape to feed into the keras model
# The 'length' or number of images you can put into the array is
# determined by the first position in the shape tuple, in this case 1
data = np.ndarray(shape=(1, 224, 224, 3), dtype=np.float32)

# Replace this with the path to your image
image = Image.open("/content/360_F_74185266_av35wpkXSsh83IwbBD8Cf2bTBvSozD4Q.jpg").convert("RGB")

# resizing the image to be at least 224x224 and then cropping from the center
size = (224, 224)
image = ImageOps.fit(image, size, Image.Resampling.LANCZOS)

# turn the image into a numpy array
image_array = np.asarray(image)

# Normalize the image
normalized_image_array = (image_array.astype(np.float32) / 127.5) - 1

# Load the image into the array
data[0] = normalized_image_array

# Predicts the model
prediction = model.predict(data)
index = np.argmax(prediction)
class_name = class_names[index]
confidence_score = prediction[0][index]

# Print prediction and confidence score
print("Class:", class_name[2:], end="")
print("Confidence Score:", confidence_score)


1/1 [==============================] - 2s 2s/step
Class: drowning
Confidence Score: 0.99375266


In [ ]:
!pip uninstall tensorflow -y


Found existing installation: tensorflow 2.18.0
Uninstalling tensorflow-2.18.0:
  Successfully uninstalled tensorflow-2.18.0


In [ ]:
!pip install tensorflow==2.12 keras==2.12


INFO: pip is looking at multiple versions of jax to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of jax to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 586.0/586.0 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 34.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 25.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.1/17.1 MB 54.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 87.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 440.7/440.7 kB 30.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0.0/78.4 kB ? eta -:--:--  Attempting uninstall: keras
    Found existing installation: keras 3.8.0
    Uninstalling keras-3.8.0:
      Successfully uninstalled keras-3.8.0
  Attempting uninstall: ga

In [ ]:
!pip install pyngrok
from pyngrok import ngrok

# Replace this with your ngrok auth token
ngrok.set_auth_token("2qAeVNj7sAoxnC1AbeLC0qpxxEY_73YbST6jPoVU3ZK4ksLbn")


In [ ]:
# Install necessary libraries
!pip install flask pyngrok tensorflow keras pillow numpy

import os
from flask import Flask, request, render_template, redirect, url_for, send_from_directory
from pyngrok import ngrok  # Use pyngrok directly
from keras.models import load_model
from PIL import Image, ImageOps
import numpy as np

# Initialize Flask app
app = Flask(__name__)

# Setup Ngrok
public_url = ngrok.connect(5000)
print(f" * ngrok tunnel \"{public_url}\" -> \"http://127.0.0.1:5000\"")

# Load the trained model (replace with the actual model file in your Colab environment)
model = load_model("/content/keras_model.h5", compile=False)

# Load the class labels
class_names = open("/content/labels.txt", "r").readlines()

# Define the upload folder to point to the Google Drive images folder
UPLOAD_FOLDER = '/content/drive/MyDrive/images/'
app.config['UPLOAD_FOLDER'] = UPLOAD_FOLDER

# Ensure the upload folder exists
if not os.path.exists(UPLOAD_FOLDER):
    os.makedirs(UPLOAD_FOLDER)

# Function to preprocess and predict the image
def predict_image(image_path):
    image = Image.open(image_path).convert("RGB")

    # Preprocess the image
    size = (224, 224)
    image = ImageOps.fit(image, size, Image.Resampling.LANCZOS)
    image_array = np.asarray(image)
    normalized_image_array = (image_array.astype(np.float32) / 127.5) - 1

    # Prepare the image data
    data = np.ndarray(shape=(1, 224, 224, 3), dtype=np.float32)
    data[0] = normalized_image_array

    # Predict using the model
    prediction = model.predict(data)
    index = np.argmax(prediction)
    class_name = class_names[index].strip()  # Clean the class name
    confidence_score = prediction[0][index]

    return class_name, confidence_score

# Route for the home page
@app.route('/')
def index():
    return '''
        <h2>ACTIVE DROWNING DETECTION</h2>
        <form action="/upload" method="post" enctype="multipart/form-data">
            <label for="file">Upload Image:</label>
            <input type="file" name="file" id="file" accept="image/*" required>
            <button type="submit">Upload and Classify</button>
        </form>
    '''

# Route to serve uploaded images
@app.route('/images/<filename>')
def uploaded_file(filename):
    return send_from_directory(app.config['UPLOAD_FOLDER'], filename)

# Route for handling file uploads
@app.route('/upload', methods=['POST'])
def upload_file():
    if 'file' not in request.files:
        return redirect(request.url)

    file = request.files['file']
    if file.filename == '':
        return redirect(request.url)

    if file:
        # Save the uploaded file to the Google Drive images folder
        file_path = os.path.join(app.config['UPLOAD_FOLDER'], file.filename)
        file.save(file_path)

        # Predict the image class and confidence
        class_name, confidence_score = predict_image(file_path)

        # Display the result along with the uploaded image
        image_url = f"/images/{file.filename}"
        return f'''
            <h2>Image Classification Result</h2>
            <img src="{image_url}" alt="Uploaded Image" width="300px">
            <p><b>Class:</b> {class_name}</p>
            <p><b>Confidence Score:</b> {confidence_score}</p>
            <p><a href="/">Go Back</a></p>
        '''

# Run the Flask app with ngrok
if __name__ == "__main__":
    app.run(host="0.0.0.0", port=5000)

 * ngrok tunnel "NgrokTunnel: "https://591b-34-138-0-144.ngrok-free.app" -> "http://localhost:5000"" -> "http://127.0.0.1:5000"


OSError: No file or directory found at /content/keras_model.h5

In [ ]:
app.config['DEBUG'] = True
app.run(host="0.0.0.0", port=5000)


 * Serving Flask app '__main__'
 * Debug mode: on


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://172.28.0.12:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug: * Restarting with stat


In [ ]:
import ipywidgets as widgets
from IPython.display import display, HTML, Image as IPImage
from keras.models import load_model
from PIL import Image, ImageOps
import numpy as np
from io import BytesIO

# Function to handle image uploading and prediction
def predict_image(change):
    # Read the image from the uploaded file
    image_data = list(change['new'].values())[0]
    image = Image.open(BytesIO(image_data['content'])).convert("RGB")

    # Display the uploaded image
    image.show()

    # Preprocess the image
    size = (224, 224)
    image = ImageOps.fit(image, size, Image.Resampling.LANCZOS)
    image_array = np.asarray(image)
    normalized_image_array = (image_array.astype(np.float32) / 127.5) - 1

    # Prepare the image data
    data = np.ndarray(shape=(1, 224, 224, 3), dtype=np.float32)
    data[0] = normalized_image_array

    # Predict using the model
    prediction = model.predict(data)
    index = np.argmax(prediction)

    # Fix class names mapping (make sure your labels file has 'Swimming' and 'Drowning')
    class_name = class_names[index].strip()  # Remove extra spaces or newlines
    confidence_score = prediction[0][index]

    # Display prediction result
    if class_name.lower() == 'swimming':
        result_label.value = f"<b>Class:</b> {class_name.capitalize()}<br><b>Confidence Score:</b> {confidence_score:.2f}<br><b>Status:</b> Swimming detected"
    elif class_name.lower() == 'drowning':
        result_label.value = f"<b>Class:</b> {class_name.capitalize()}<br><b>Confidence Score:</b> {confidence_score:.2f}<br><b>Status:</b> Drowning detected"
    else:
        result_label.value = f"<b>Class:</b> {class_name.capitalize()}<br><b>Confidence Score:</b> {confidence_score:.2f}<br><b>Status:</b> Unknown classification"

    result_label.layout.visibility = 'visible'  # Make result visible after prediction

# Load the model
model = load_model("keras_model.h5", compile=False)

# Load the labels (Make sure 'labels.txt' has the correct labels like 'Swimming' and 'Drowning')
class_names = open("labels.txt", "r").readlines()

# Set up the widgets
upload_button = widgets.FileUpload(accept="image/*", multiple=False)
upload_button.observe(predict_image, names='value')

# Label to show prediction results
result_label = widgets.HTML(value="<i>Prediction will appear here.</i>", layout=widgets.Layout(visibility='hidden'))

# Style for widgets
upload_button.style.button_color = 'blue'
upload_button.layout.width = '300px'

result_label.layout.margin = '20px 0 0 0'
result_label.layout.padding = '10px'
result_label.layout.background_color = '#f2f2f2'
result_label.layout.border = '1px solid #ddd'
result_label.layout.width = '300px'
result_label.layout.text_align = 'center'

# Heading for the interface
header = widgets.HTML(value="<h2 style='color:#4CAF50;'>Image Classification Using Keras Model</h2>")

# Display the widgets with heading
display(header, upload_button, result_label)


HTML(value="<h2 style='color:#4CAF50;'>Image Classification Using Keras Model</h2>")

FileUpload(value={}, accept='image/*', description='Upload', layout=Layout(width='300px'), style=ButtonStyle(b…

HTML(value='<i>Prediction will appear here.</i>', layout=Layout(border='1px solid #ddd', margin='20px 0 0 0', …

In [ ]:
# Install necessary libraries
!pip install flask pyngrok tensorflow keras pillow numpy

import os
from flask import Flask, request, render_template, redirect, url_for, send_from_directory
from pyngrok import ngrok  # Use pyngrok directly
from keras.models import load_model
from PIL import Image, ImageOps
import numpy as np

# Initialize Flask app
app = Flask(__name__)

# Setup Ngrok
public_url = ngrok.connect(5000)
print(f" * ngrok tunnel \"{public_url}\" -> \"http://127.0.0.1:5000\"")

# Load the trained model (replace with the actual model file in your Colab environment)
model = load_model("/content/yolov3_model.h5", compile=False)

# Load the class labels
class_names = open("/content/labels.txt", "r").readlines()

# Define the upload folder to point to the Google Drive images folder
UPLOAD_FOLDER = '/content/drive/MyDrive/images/'
app.config['UPLOAD_FOLDER'] = UPLOAD_FOLDER

# Ensure the upload folder exists
if not os.path.exists(UPLOAD_FOLDER):
    os.makedirs(UPLOAD_FOLDER)

# Function to preprocess and predict the image
def predict_image(image_path):
    image = Image.open(image_path).convert("RGB")

    # Preprocess the image
    size = (224, 224)
    image = ImageOps.fit(image, size, Image.Resampling.LANCZOS)
    image_array = np.asarray(image)
    normalized_image_array = (image_array.astype(np.float32) / 127.5) - 1

    # Prepare the image data
    data = np.ndarray(shape=(1, 224, 224, 3), dtype=np.float32)
    data[0] = normalized_image_array

    # Predict using the model
    prediction = model.predict(data)
    index = np.argmax(prediction)
    class_name = class_names[index].strip()  # Clean the class name
    confidence_score = prediction[0][index]

    return class_name, confidence_score

# Route for the home page
@app.route('/')
def index():
    return '''
        <div style="text-align: center; font-family: Arial, sans-serif; background-color: #f0f8ff; padding: 50px; border: 3px solid #000; border-radius: 15px; width: 60%; margin: auto;">
            <h2 style="color: #333; margin-bottom: 30px;">ACTIVE DROWNING DETECTION</h2>
            <form action="/upload" method="post" enctype="multipart/form-data" style="display: flex; flex-direction: column; align-items: center; gap: 20px;">
                <div style="display: flex; gap: 10px; align-items: center;">
                    <label for="file" style="font-size: 18px; color: #555;">Upload Image:</label>
                    <input type="file" name="file" id="file" accept="image/*" required style="padding: 10px; font-size: 16px;">
                </div>
                <button type="submit" style="margin-top: 20px; padding: 10px 30px; font-size: 16px; background-color: #007BFF; color: white; border: none; border-radius: 5px;">Upload and Classify</button>
            </form>
        </div>
    '''

# Route to serve uploaded images
@app.route('/images/<filename>')
def uploaded_file(filename):
    return send_from_directory(app.config['UPLOAD_FOLDER'], filename)

# Route for handling file uploads
@app.route('/upload', methods=['POST'])
def upload_file():
    if 'file' not in request.files:
        return redirect(request.url)

    file = request.files['file']
    if file.filename == '':
        return redirect(request.url)

    if file:
        # Save the uploaded file to the Google Drive images folder
        file_path = os.path.join(app.config['UPLOAD_FOLDER'], file.filename)
        file.save(file_path)

        # Predict the image class and confidence
        class_name, confidence_score = predict_image(file_path)

        # Display the result along with the uploaded image
        image_url = f"/images/{file.filename}"
        return f'''
            <div style="text-align: center; font-family: Arial, sans-serif; background-color: #f0f8ff; padding: 50px; border: 3px solid #000; border-radius: 15px; width: 60%; margin: auto;">
                <h2 style="color: #333;">Image Classification Result</h2>
                <img src="{image_url}" alt="Uploaded Image" width="300px" style="margin: 20px; border: 1px solid #ccc; border-radius: 5px;">
                <p style="font-size: 18px; color: #555;"><b>Class:</b> {class_name}</p>
                <p style="font-size: 18px; color: #555;"><b>Confidence Score:</b> {confidence_score:.2f}</p>
                <p><a href="/" style="color: #007BFF; text-decoration: none;">Go Back</a></p>
            </div>
        '''

# Run the Flask app with ngrok
if __name__ == "__main__":
    app.run(host="0.0.0.0", port=5000)


 * ngrok tunnel "NgrokTunnel: "https://5f95-34-138-0-144.ngrok-free.app" -> "http://localhost:5000"" -> "http://127.0.0.1:5000"
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://172.28.0.12:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [22/Feb/2025 09:07:53] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [22/Feb/2025 09:07:54] "GET /favicon.ico HTTP/1.1" 404 -


1/1 [==============================] - 1s 985ms/step


INFO:werkzeug:127.0.0.1 - - [22/Feb/2025 09:08:04] "POST /upload HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [22/Feb/2025 09:08:04] "GET /images/360_F_74185266_av35wpkXSsh83IwbBD8Cf2bTBvSozD4Q.jpg HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [22/Feb/2025 09:11:36] "GET / HTTP/1.1" 200 -


1/1 [==============================] - 0s 44ms/step


INFO:werkzeug:127.0.0.1 - - [22/Feb/2025 09:12:06] "POST /upload HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [22/Feb/2025 09:12:06] "GET /images/360_F_74185266_av35wpkXSsh83IwbBD8Cf2bTBvSozD4Q.jpg HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [22/Feb/2025 09:15:18] "GET / HTTP/1.1" 200 -


1/1 [==============================] - 0s 39ms/step


INFO:werkzeug:127.0.0.1 - - [22/Feb/2025 09:15:30] "POST /upload HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [22/Feb/2025 09:15:31] "GET /images/71d0d72ac35dd779554baa4e5e68c1be48cb0b5a_HP_ClassesDetail_HeaderOTHERS_0000s_0003s_0003_iStock_502.jpg HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [22/Feb/2025 09:15:55] "GET / HTTP/1.1" 200 -


In [ ]:
!ngrok authtoken 2qAeVNj7sAoxnC1AbeLC0qpxxEY_73YbST6jPoVU3ZK4ksLbn


Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
# Install necessary libraries
!pip install flask pyngrok tensorflow keras pillow numpy

import os
from flask import Flask, request, render_template, redirect, url_for, send_from_directory
from pyngrok import ngrok  # Use pyngrok directly
from keras.models import load_model
from PIL import Image, ImageOps
import numpy as np

# Initialize Flask app
app = Flask(__name__)

# Setup Ngrok
public_url = ngrok.connect(5000)
print(f" * ngrok tunnel \"{public_url}\" -> \"http://127.0.0.1:5000\"")

# Load the trained model (replace with the actual model file in your Colab environment)
model = load_model("keras_model.h5", compile=False)

# Load the class labels
class_names = open("labels.txt", "r").readlines()

# Define the upload folder to point to the Google Drive images folder
UPLOAD_FOLDER = '/content/drive/MyDrive/images/'
app.config['UPLOAD_FOLDER'] = UPLOAD_FOLDER

# Ensure the upload folder exists
if not os.path.exists(UPLOAD_FOLDER):
    os.makedirs(UPLOAD_FOLDER)

# Function to preprocess and predict the image
def predict_image(image_path):
    image = Image.open(image_path).convert("RGB")

    # Preprocess the image
    size = (224, 224)
    image = ImageOps.fit(image, size, Image.Resampling.LANCZOS)
    image_array = np.asarray(image)
    normalized_image_array = (image_array.astype(np.float32) / 127.5) - 1

    # Prepare the image data
    data = np.ndarray(shape=(1, 224, 224, 3), dtype=np.float32)
    data[0] = normalized_image_array

    # Predict using the model
    prediction = model.predict(data)
    index = np.argmax(prediction)
    class_name = class_names[index].strip()  # Clean the class name
    confidence_score = prediction[0][index]

    return class_name, confidence_score

# Route for the home page
@app.route('/')
def index():
    return '''
        <h2>Image Classification Using Keras Model</h2>
        <form action="/upload" method="post" enctype="multipart/form-data">
            <label for="file">Upload Image:</label>
            <input type="file" name="file" id="file" accept="image/*" required>
            <button type="submit">Upload and Classify</button>
        </form>
    '''

# Route to serve uploaded images
@app.route('/images/<filename>')
def uploaded_file(filename):
    return send_from_directory(app.config['UPLOAD_FOLDER'], filename)

# Route for handling file uploads
@app.route('/upload', methods=['POST'])
def upload_file():
    if 'file' not in request.files:
        return redirect(request.url)

    file = request.files['file']
    if file.filename == '':
        return redirect(request.url)

    if file:
        # Save the uploaded file to the Google Drive images folder
        file_path = os.path.join(app.config['UPLOAD_FOLDER'], file.filename)
        file.save(file_path)

        # Predict the image class and confidence
        class_name, confidence_score = predict_image(file_path)

        # Display the result along with the uploaded image
        image_url = f"/images/{file.filename}"
        return f'''
            <h2>Image Classification Result</h2>
            <img src="{image_url}" alt="Uploaded Image" width="300px">
            <p><b>Class:</b> {class_name}</p>
            <p><b>Confidence Score:</b> {confidence_score}</p>
            <p><a href="/">Go Back</a></p>
        '''

# Run the Flask app with ngrok
if __name__ == "__main__":
    app.run()


 * ngrok tunnel "NgrokTunnel: "https://82e7-34-125-221-81.ngrok-free.app" -> "http://localhost:5000"" -> "http://127.0.0.1:5000"
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive
